## Importing Necessary Libraries and Loading Data

In [1]:
import json
import pandas as pd
import numpy as np
import pickle
import nltk
import re
from nltk.corpus import stopwords
from collections import Counter
from time import time

In [2]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 150)

In [3]:
pickle_in = open(r"C:\Users\surya\Desktop\Jupyter Python Programs\Amazon Review NLP\clean_cpa_data_excel.pickle", 'rb')
df = pickle.load(pickle_in)
df

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image,description,feature,rank,main_cat,imageURL,imageURLHighRes
0,5,True,"08 4, 2014",A24E3SXTC62LJI,7508492919,{'Color:': ' Bling'},Claudia Valdivia,Looks even better in person. Be careful to not...,Can't stop won't stop looking at it,1407110400,NaN,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
1,5,True,"02 12, 2014",A269FLZCB4GIPV,7508492919,NaN,sarah ponce,When you don't want to spend a whole lot of ca...,1,1392163200,NaN,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
2,3,True,"02 8, 2014",AB6CHQWHZW4TV,7508492919,NaN,Kai,"so the case came on time, i love the design. I...",Its okay,1391817600,NaN,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
3,2,True,"02 4, 2014",A1M117A53LEI8,7508492919,NaN,Sharon Williams,DON'T CARE FOR IT. GAVE IT AS A GIFT AND THEY...,CASE,1391472000,NaN,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
4,4,True,"02 3, 2014",A272DUT8M88ZS8,7508492919,NaN,Bella Rodriguez,"I liked it because it was cute, but the studs ...",Cute!,1391385600,NaN,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1041972,5,True,"10 3, 2016",A3Q7GA2Q62E18Q,B01HE0RNYO,NaN,Kerry Moore,Excellent product for the price!!!,Inexpensive but good quality...,1475452800,NaN,NaN,['<b>Compatible with Apple iPhone 6 Plus and i...,['[Compatible with Apple iPhone 6 Plus and iPh...,"['>#574,746 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
1041973,5,False,"07 18, 2016",AEKOIX49JGCUG,B01HE0RNYO,NaN,Wishart,I needed a new phone case for my iPhone 6 plus...,Great Case. Gonna Buy Again.,1468800000,NaN,NaN,['<b>Compatible with Apple iPhone 6 Plus and i...,['[Compatible with Apple iPhone 6 Plus and iPh...,"['>#574,746 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
1041974,4,False,"07 16, 2016",A2KOKSKEDIBRGQ,B01HE0RNYO,NaN,Wise One,The covers my iPhone just fine. It is simple i...,Plain Jane of iPhone Covers,1468627200,NaN,NaN,['<b>Compatible with Apple iPhone 6 Plus and i...,['[Compatible with Apple iPhone 6 Plus and iPh...,"['>#574,746 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
1041975,3,False,"07 11, 2016",A1BO3NH4KKASUT,B01HE0RNYO,NaN,Bebescastle,This free case is okay it slim and trendy look...,Its easy to hold and not bulky,1468195200,NaN,NaN,['<b>Compatible with Apple iPhone 6 Plus and i...,['[Compatible with Apple iPhone 6 Plus and iPh...,"['>#574,746 in 

## Task 1 - Replacing NaN values in Vote column with 0

In [4]:
df['vote'].replace(np.nan, 0, inplace=True)

## Task 2 - Removing rows which have NaN values in reviewText column

In [5]:
df['reviewText'].isna().sum()

708

In [6]:
df[df['reviewText'].isna() == True]

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image,description,feature,rank,main_cat,imageURL,imageURLHighRes
10371,1,True,"08 17, 2014",A11NWZU48ILO52,B001ETRNAQ,NaN,Sherry Harrison,NaN,One Star,1408233600,0,NaN,['Product Description\nDeluxe SmartClip Univer...,['Keep your cell phone or iPod/MP3 player at h...,"['>#135,811 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
15635,5,True,"03 13, 2018",A1T5PIZ3B56C4O,B0027HNIFS,NaN,Donna Rhoades,NaN,Five Stars,1520899200,0,NaN,"[""The Barely There is Case-mate's new line of ...","['Extremely slim profile', 'Impact resistant f...","['>#798,937 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
17178,5,True,"09 14, 2016",A22LWUQ7I5V0TU,B002BSO4ZG,NaN,Frandchesta ransby,NaN,Five Stars,1473811200,0,NaN,['Technical details easily open/remove/eject t...,"['Easily open/remove/eject the sim card tray',...","['>#41,729 in Cell Phones & Accessories (See T...",Cell Phones & Accessories,[],[]
18586,5,True,"08 11, 2015",AYQKPQE4QBP4C,B002NGEFSC,NaN,Gilberto Quiroga Trevino,NaN,Five Stars,1439251200,0,NaN,"[""Your iphone is as attractive and usable whil...",['Keep it slim on your pocket and this case ca...,"['>#415,465 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
24541,5,True,"11 29, 2014",A13CHSE3IU391N,B003RDV0VU,"{'Color:': ' Clear', 'Package Type:': ' Standa...",Michele,NaN,Five Stars,1417219200,0,NaN,['Generic Combo Screen Protector for HTC EVO 4...,"['Htc desire hd clear screen protector', 'good...","['>#174,009 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1037197,5,True,"07 15, 2017",A1E7Y6S5G0GG2L,B01HEX7ZNK,{'Color:': ' Black'},Countryboy46,NaN,Five Stars,1500076800,0,NaN,[],"['PRODUCTS COMBINATION: 3 Pack, the lengths ar...","['>#25,218 in Cell Phones & Accessories (See T...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
1038552,4,True,"06 3, 2016",A283C2JOPY8MOE,B00CKZI00A,NaN,Jose Alberto Cordero Peralta,NaN,Four Stars,1464912000,0,NaN,['First impression is the last impression and ...,['Cellular Band - Quad-Band 850 / 900 / 1800 /...,"['>#1,287,690 in Cell Phones & Accessories (Se...",Cell Phones & Accessories,[],[]
1038847,4,True,"03 27, 2015",A1TE0PQF0RCYG0,B00G91JSPM,NaN,1234,NaN,Four Stars,1427414400,0,NaN,['Color: Black <br>\n\nUse the car with a wind...,[],"['>#1,260,488 in Cell Phones & Accessories (Se...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
1040987,5,True,"07 25, 2016",A1KNCPY68RESBS,B017NHZDSG,NaN,silas,NaN,Five Stars,1469404800,0,NaN,[],[],"['>#550,155 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,[],[]


In [7]:
df.dropna(subset = ['reviewText'], inplace=True)

## Task 3 - Removing rows which have numbers in reviewText column

In [8]:
df[df['reviewText'].astype(str).str.isdigit()]

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image,description,feature,rank,main_cat,imageURL,imageURLHighRes
174,5,True,"05 18, 2015",A2LBWVL4ZW0YMU,8288878881,NaN,Cynthia Poole,1,Five Stars,1431907200,0,NaN,"[""Safely charge your phone using the original ...",['Safely charge your phone using the original ...,"['>#245,551 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,[],[]
11964,4,False,"12 30, 2008",A3TXQIOHXRCDFC,B001JT1U1E,NaN,Riyad Kalla,0,"Excellent Google Smart Phone, Good General Sma...",1230595200,193,NaN,"['Google HTC Dream G1, Google Phone, Unlocked'...","[""Google Android OS-powered smartphone in blac...","['>#1,523,869 in Cell Phones & Accessories (Se...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
25664,5,True,"12 24, 2015",A1J8PKJMGO2JQ6,B003SX0OW0,"{'Size:': ' One Size', 'Color:': ' black/grey'...",stildreamn,1,Five Stars,1450915200,0,NaN,"[""The SILICRYLIC by Incipio is an innovative 2...","['Compatible with iPhone 4 / iPhone 4s', 'Inno...","['>#291,321 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
39243,3,True,"04 13, 2015",AVQV7CFXRJ2T2,B004E30BJO,NaN,Nicole G.,1,Three Stars,1428883200,0,NaN,"['Pink iPhone 4 / 4S Rim Bumper, case, cover w...","['Length x Width x Thickness: 4.75"" x 2.5"" x 0...","['>#377,077 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
42033,2,True,"04 13, 2015",AVQV7CFXRJ2T2,B004I53BOA,{'Color:': ' Silver'},Nicole G.,1,Two Stars,1428883200,0,NaN,['This slim fit snap-on case compatible with A...,['This slim fit snap-on case for Apple iPhone ...,"['>#255,288 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
48963,2,True,"04 13, 2015",AVQV7CFXRJ2T2,B004TSL846,"{'Color:': ' Smoke', 'Package Type:': ' Standa...",Nicole G.,1,Two Stars,1428883200,0,NaN,['This Asamyna Zebra Slim and Durable Protecti...,['Showcase your own personal style while maint...,"['>#386,821 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
54650,5,True,"05 18, 2015",A2LBWVL4ZW0YMU,B004ZSGAMU,NaN,Cynthia Poole,1,Five Stars,1431907200,0,NaN,"['', ""The Motorola Droid X is a high-end Andro...",['The Motorola Droid X is an Android device sp...,"['>#165,401 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
62520,5,True,"08 18, 2015",A2H3X3KW00LMQJ,B0059DLLJC,NaN,Timbobillybob,1,Five Stars,1439856000,0,NaN,['This snap-on rubber coated case keeps your A...,['This snap-on rubber coated case keeps your A...,"['>#140,311 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
63215,3,True,"04 13, 2015",AVQV7CFXRJ2T2,B005AAXPT8,NaN,Nicole G.,1,Three Stars,1428883200,0,NaN,['Classic bichrome bumper skin/cover exclusive...,"['Length x Width x Thickness: 4.75"" x 2.5"" x 0...","['>#503,012 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
63310,2,True,"04 13, 2015",AVQV7CFXRJ2T2,B005AAREUO,NaN,Nicole G.,1,Two Stars,1428883200,0,NaN,['White and Blue Premium Bumper Case for Apple...,"['Length x Width x Thickness: 4.75"" x 2.5"" x 0...","['>#210,979 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,[],[]


In [9]:
number_list = list(df[df['reviewText'].astype(str).str.isdigit()].index)

In [10]:
len(number_list)

100

In [11]:
df.drop(number_list, inplace=True)

## Task 4 - Resetting index, storing rows indices in a variable 'rows' for future use, and viewing the final dataframe

In [12]:
df.reset_index(drop=True, inplace=True)

In [13]:
rows = list(df.index)
rows

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [14]:
df

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image,description,feature,rank,main_cat,imageURL,imageURLHighRes
0,5,True,"08 4, 2014",A24E3SXTC62LJI,7508492919,{'Color:': ' Bling'},Claudia Valdivia,Looks even better in person. Be careful to not...,Can't stop won't stop looking at it,1407110400,0,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
1,5,True,"02 12, 2014",A269FLZCB4GIPV,7508492919,NaN,sarah ponce,When you don't want to spend a whole lot of ca...,1,1392163200,0,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
2,3,True,"02 8, 2014",AB6CHQWHZW4TV,7508492919,NaN,Kai,"so the case came on time, i love the design. I...",Its okay,1391817600,0,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
3,2,True,"02 4, 2014",A1M117A53LEI8,7508492919,NaN,Sharon Williams,DON'T CARE FOR IT. GAVE IT AS A GIFT AND THEY...,CASE,1391472000,0,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
4,4,True,"02 3, 2014",A272DUT8M88ZS8,7508492919,NaN,Bella Rodriguez,"I liked it because it was cute, but the studs ...",Cute!,1391385600,0,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1041164,5,True,"10 3, 2016",A3Q7GA2Q62E18Q,B01HE0RNYO,NaN,Kerry Moore,Excellent product for the price!!!,Inexpensive but good quality...,1475452800,0,NaN,['<b>Compatible with Apple iPhone 6 Plus and i...,['[Compatible with Apple iPhone 6 Plus and iPh...,"['>#574,746 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
1041165,5,False,"07 18, 2016",AEKOIX49JGCUG,B01HE0RNYO,NaN,Wishart,I needed a new phone case for my iPhone 6 plus...,Great Case. Gonna Buy Again.,1468800000,0,NaN,['<b>Compatible with Apple iPhone 6 Plus and i...,['[Compatible with Apple iPhone 6 Plus and iPh...,"['>#574,746 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
1041166,4,False,"07 16, 2016",A2KOKSKEDIBRGQ,B01HE0RNYO,NaN,Wise One,The covers my iPhone just fine. It is simple i...,Plain Jane of iPhone Covers,1468627200,0,NaN,['<b>Compatible with Apple iPhone 6 Plus and i...,['[Compatible with Apple iPhone 6 Plus and iPh...,"['>#574,746 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...
1041167,3,False,"07 11, 2016",A1BO3NH4KKASUT,B01HE0RNYO,NaN,Bebescastle,This free case is okay it slim and trendy look...,Its easy to hold and not bulky,1468195200,0,NaN,['<b>Compatible with Apple iPhone 6 Plus and i...,['[Compatible with Apple iPhone 6 Plus and iPh...,"['>#574,746 in Cell Phones & Acce

## Task 5 - Calculating number of sentences in reviewText (before removing stopwords and punctuation marks, as if fullstop eliminated calculation won't be possible)

###  Function for getting the number of sentences in a review

In [15]:
def sent_len(sentence):
    return len(nltk.sent_tokenize(sentence))

### Example

In [16]:
for i in range(5):
    print(sent_len(df.iloc[i]['reviewText']))

4
1
3
3
2


In [17]:
sent_list1 = [sent_len(df.iloc[i]['reviewText']) for i in range(5)]

In [18]:
sent_list1

[4, 1, 3, 3, 2]

### Applying this on main review data

In [19]:
sent_list = [sent_len(df.iloc[i]['reviewText']) for i in rows]

In [20]:
sent_list

[4,
 1,
 3,
 3,
 2,
 3,
 7,
 5,
 6,
 2,
 1,
 2,
 2,
 3,
 2,
 3,
 3,
 2,
 3,
 3,
 2,
 4,
 1,
 3,
 2,
 4,
 5,
 1,
 6,
 3,
 1,
 3,
 1,
 4,
 3,
 9,
 4,
 6,
 2,
 3,
 4,
 1,
 3,
 4,
 3,
 1,
 2,
 2,
 2,
 2,
 3,
 2,
 3,
 5,
 1,
 3,
 5,
 4,
 9,
 2,
 5,
 2,
 6,
 1,
 1,
 4,
 3,
 3,
 2,
 2,
 6,
 3,
 4,
 4,
 3,
 4,
 3,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 6,
 2,
 1,
 4,
 4,
 1,
 1,
 2,
 1,
 4,
 4,
 3,
 5,
 5,
 2,
 2,
 4,
 3,
 11,
 3,
 3,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 2,
 4,
 2,
 1,
 1,
 1,
 2,
 4,
 1,
 2,
 7,
 1,
 2,
 8,
 2,
 5,
 2,
 5,
 4,
 3,
 1,
 2,
 3,
 1,
 1,
 3,
 2,
 1,
 3,
 5,
 1,
 1,
 4,
 4,
 3,
 2,
 2,
 1,
 3,
 1,
 1,
 1,
 10,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 4,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 3,
 3,
 3,
 3,
 3,
 2,
 2,
 2,
 2,
 4,
 2,
 2,
 1,
 4,
 2,
 2,
 3,
 3,
 3,
 4,
 2,
 3,
 11,
 3,
 4,
 1,
 4,
 4,
 2,
 2,
 2,
 3,
 3,
 2,
 1,
 1,
 5,
 2,
 2,
 7,
 2,
 2,
 2,
 4,
 2,
 1,
 2,
 4,
 1,
 4,
 3,
 4,
 11,


In [21]:
df['no_of_sent'] = sent_list

In [22]:
df.head()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image,description,feature,rank,main_cat,imageURL,imageURLHighRes,no_of_sent
0,5,True,"08 4, 2014",A24E3SXTC62LJI,7508492919,{'Color:': ' Bling'},Claudia Valdivia,Looks even better in person. Be careful to not...,Can't stop won't stop looking at it,1407110400,0,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,4
1,5,True,"02 12, 2014",A269FLZCB4GIPV,7508492919,NaN,sarah ponce,When you don't want to spend a whole lot of ca...,1,1392163200,0,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,1
2,3,True,"02 8, 2014",AB6CHQWHZW4TV,7508492919,NaN,Kai,"so the case came on time, i love the design. I...",Its okay,1391817600,0,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,3
3,2,True,"02 4, 2014",A1M117A53LEI8,7508492919,NaN,Sharon Williams,DON'T CARE FOR IT. GAVE IT AS A GIFT AND THEY...,CASE,1391472000,0,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,3
4,4,True,"02 3, 2014",A272DUT8M88ZS8,7508492919,NaN,Bella Rodriguez,"I liked it because it was cute, but the studs ...",Cute!,1391385600,0,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,2


## Task 6 - Removing stopwords, numbers and punctuation marks in reviewText, i.e. keeping only a-z and A-Z characters, followed by calculating number of words and unique words

### Approach for cleaning review

In [23]:
df.iloc[0]['reviewText']

'Looks even better in person. Be careful to not drop your phone so often because the rhinestones will fall off (duh). More of a decorative case than it is protective, but I will say that it fits perfectly and securely on my phone. Overall, very pleased with this purchase.'

In [24]:
a = re.sub('[^a-zA-z]', ' ', df.iloc[0]['reviewText'])  # removing everything which is not in a to z and A to Z, i.e. numbers, comma, apostrophe,etc.
a

'Looks even better in person  Be careful to not drop your phone so often because the rhinestones will fall off  duh   More of a decorative case than it is protective  but I will say that it fits perfectly and securely on my phone  Overall  very pleased with this purchase '

In [25]:
a = a.lower()
a

'looks even better in person  be careful to not drop your phone so often because the rhinestones will fall off  duh   more of a decorative case than it is protective  but i will say that it fits perfectly and securely on my phone  overall  very pleased with this purchase '

In [26]:
a = a.split()  # Since punctuation marks removed, split() and word_tokenize() can both be used, but split() is faster so we go with that
a

['looks',
 'even',
 'better',
 'in',
 'person',
 'be',
 'careful',
 'to',
 'not',
 'drop',
 'your',
 'phone',
 'so',
 'often',
 'because',
 'the',
 'rhinestones',
 'will',
 'fall',
 'off',
 'duh',
 'more',
 'of',
 'a',
 'decorative',
 'case',
 'than',
 'it',
 'is',
 'protective',
 'but',
 'i',
 'will',
 'say',
 'that',
 'it',
 'fits',
 'perfectly',
 'and',
 'securely',
 'on',
 'my',
 'phone',
 'overall',
 'very',
 'pleased',
 'with',
 'this',
 'purchase']

#### Note - Above steps can be done directly by re.sub(r"[^a-zA-Z]", " ", df.iloc[0]['reviewText'].lower()).split()

In [27]:
a = [word for word in a if not word in set(stopwords.words('english'))]
a

['looks',
 'even',
 'better',
 'person',
 'careful',
 'drop',
 'phone',
 'often',
 'rhinestones',
 'fall',
 'duh',
 'decorative',
 'case',
 'protective',
 'say',
 'fits',
 'perfectly',
 'securely',
 'phone',
 'overall',
 'pleased',
 'purchase']

### Calculating number of words and unique words

In [28]:
len(a)        # number of words

22

In [29]:
len(set(a))   # number of unique words ('phone' is repeated)

21

### Function for cleaning a review and returning the number of words and unique words in it

In [30]:
cachedStopWords = set(stopwords.words("english"))  # storing the stopwords in a variable first speeds up the process
stopwords_dict = Counter(cachedStopWords)  # Ref - https://stackoverflow.com/questions/19560498/faster-way-to-remove-stop-words-in-python

In [31]:
def clean(sentence):
    a = re.sub(r"[^a-zA-Z]", " ", sentence.lower()).split()
    a = [word for word in a if not word in stopwords_dict]
    return len(a), len(set(a))

### Example

In [32]:
clean(df.iloc[0]['reviewText'])

(22, 21)

In [33]:
clean(df.iloc[0]['reviewText'])[0]

22

In [34]:
clean(df.iloc[0]['reviewText'])[1]

21

### Example

In [35]:
word_list1 = [clean(df.iloc[i]['reviewText'])[0] for i in range(5)]

In [36]:
word_list1

[22, 10, 22, 5, 12]

In [37]:
unique_word_list1 = [clean(df.iloc[i]['reviewText'])[1] for i in range(5)]

In [38]:
unique_word_list1

[21, 9, 21, 5, 12]

In [39]:
b = df.iloc[8]['reviewText']  # 1, 3, 4!, 6, 8!
b

"DO NOT BUY! this item is seriously cheap as heck. not worth buying it at all. I didn't even get to use it and it was already losing all of its gems. I wish I got my money back on this item!!"

In [40]:
a = re.sub(r"[^a-zA-Z]", " ", b.lower()).split()
a = [word for word in a if not word in set(stopwords.words('english'))]
a

['buy',
 'item',
 'seriously',
 'cheap',
 'heck',
 'worth',
 'buying',
 'even',
 'get',
 'use',
 'already',
 'losing',
 'gems',
 'wish',
 'got',
 'money',
 'back',
 'item']

In [41]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

### Applying on main review data

In [42]:
start = time()

In [43]:
word_list = [clean(df.iloc[i]['reviewText'])[0] for i in rows]

In [44]:
end = time()
end - start

126.46220374107361

In [45]:
word_list

[22,
 10,
 22,
 5,
 12,
 14,
 31,
 27,
 18,
 18,
 13,
 13,
 10,
 9,
 12,
 14,
 12,
 12,
 14,
 12,
 15,
 15,
 11,
 11,
 23,
 12,
 21,
 9,
 19,
 14,
 17,
 22,
 9,
 14,
 13,
 66,
 15,
 24,
 12,
 10,
 18,
 8,
 35,
 21,
 11,
 17,
 12,
 10,
 10,
 13,
 13,
 10,
 17,
 23,
 12,
 11,
 15,
 41,
 52,
 11,
 21,
 12,
 40,
 1,
 3,
 14,
 13,
 12,
 30,
 14,
 23,
 17,
 14,
 22,
 12,
 22,
 21,
 1,
 2,
 1,
 6,
 3,
 17,
 14,
 12,
 9,
 14,
 12,
 37,
 24,
 16,
 16,
 24,
 2,
 2,
 9,
 5,
 31,
 18,
 10,
 16,
 23,
 16,
 11,
 17,
 16,
 63,
 13,
 10,
 10,
 10,
 1,
 1,
 1,
 2,
 2,
 2,
 6,
 5,
 5,
 2,
 1,
 4,
 5,
 14,
 7,
 1,
 8,
 4,
 13,
 12,
 12,
 10,
 70,
 23,
 21,
 63,
 8,
 46,
 14,
 20,
 40,
 20,
 12,
 11,
 12,
 13,
 10,
 11,
 22,
 18,
 11,
 22,
 10,
 14,
 16,
 12,
 12,
 24,
 16,
 2,
 5,
 2,
 2,
 1,
 65,
 2,
 6,
 2,
 2,
 1,
 1,
 1,
 3,
 15,
 1,
 1,
 2,
 2,
 5,
 11,
 13,
 4,
 1,
 1,
 15,
 5,
 2,
 1,
 6,
 5,
 2,
 2,
 2,
 1,
 5,
 18,
 13,
 7,
 13,
 17,
 11,
 21,
 15,
 18,
 36,
 13,
 15,
 13,
 8,
 11,
 4,
 11,
 18,

In [46]:
start = time()

In [47]:
unique_word_list = [clean(df.iloc[i]['reviewText'])[1] for i in rows]

In [48]:
end = time()
end - start

131.57319736480713

In [49]:
unique_word_list

[21,
 9,
 21,
 5,
 12,
 13,
 28,
 26,
 17,
 17,
 13,
 13,
 10,
 9,
 11,
 14,
 12,
 12,
 14,
 12,
 13,
 12,
 9,
 11,
 21,
 11,
 19,
 9,
 16,
 13,
 14,
 21,
 9,
 14,
 12,
 55,
 15,
 23,
 8,
 8,
 16,
 7,
 25,
 19,
 9,
 14,
 12,
 10,
 9,
 12,
 13,
 10,
 16,
 20,
 12,
 9,
 14,
 39,
 49,
 10,
 18,
 11,
 36,
 1,
 3,
 11,
 13,
 11,
 26,
 13,
 22,
 15,
 13,
 22,
 12,
 22,
 19,
 1,
 2,
 1,
 5,
 3,
 16,
 13,
 11,
 9,
 13,
 10,
 32,
 19,
 15,
 13,
 23,
 2,
 2,
 9,
 5,
 30,
 17,
 10,
 16,
 18,
 15,
 11,
 16,
 16,
 57,
 13,
 10,
 10,
 10,
 1,
 1,
 1,
 2,
 2,
 2,
 5,
 5,
 5,
 2,
 1,
 4,
 5,
 14,
 7,
 1,
 8,
 4,
 13,
 12,
 11,
 10,
 54,
 19,
 21,
 55,
 8,
 41,
 14,
 18,
 39,
 18,
 11,
 11,
 11,
 12,
 10,
 10,
 20,
 17,
 10,
 17,
 10,
 13,
 14,
 11,
 10,
 22,
 16,
 2,
 5,
 2,
 2,
 1,
 44,
 2,
 6,
 2,
 2,
 1,
 1,
 1,
 3,
 14,
 1,
 1,
 2,
 2,
 5,
 11,
 13,
 4,
 1,
 1,
 13,
 5,
 2,
 1,
 6,
 5,
 2,
 2,
 2,
 1,
 5,
 17,
 12,
 7,
 12,
 17,
 11,
 19,
 15,
 16,
 34,
 13,
 14,
 12,
 8,
 11,
 4,
 8,
 16,
 14,
 1

In [50]:
df['no_of_words'] = word_list
df['no_of_unique_words'] = unique_word_list

In [51]:
df

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image,description,feature,rank,main_cat,imageURL,imageURLHighRes,no_of_sent,no_of_words,no_of_unique_words
0,5,True,"08 4, 2014",A24E3SXTC62LJI,7508492919,{'Color:': ' Bling'},Claudia Valdivia,Looks even better in person. Be careful to not...,Can't stop won't stop looking at it,1407110400,0,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,4,22,21
1,5,True,"02 12, 2014",A269FLZCB4GIPV,7508492919,NaN,sarah ponce,When you don't want to spend a whole lot of ca...,1,1392163200,0,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,1,10,9
2,3,True,"02 8, 2014",AB6CHQWHZW4TV,7508492919,NaN,Kai,"so the case came on time, i love the design. I...",Its okay,1391817600,0,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,3,22,21
3,2,True,"02 4, 2014",A1M117A53LEI8,7508492919,NaN,Sharon Williams,DON'T CARE FOR IT. GAVE IT AS A GIFT AND THEY...,CASE,1391472000,0,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,3,5,5
4,4,True,"02 3, 2014",A272DUT8M88ZS8,7508492919,NaN,Bella Rodriguez,"I liked it because it was cute, but the studs ...",Cute!,1391385600,0,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,2,12,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1041164,5,True,"10 3, 2016",A3Q7GA2Q62E18Q,B01HE0RNYO,NaN,Kerry Moore,Excellent product for the price!!!,Inexpensive but good quality...,1475452800,0,NaN,['<b>Compatible with Apple iPhone 6 Plus and i...,['[Compatible with Apple iPhone 6 Plus and iPh...,"['>#574,746 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,2,3,3
1041165,5,False,"07 18, 2016",AEKOIX49JGCUG,B01HE0RNYO,NaN,Wishart,I needed a new phone case for my iPhone 6 plus...,Great Case. Gonna Buy Again.,1468800000,0,NaN,['<b>Compatible with Apple iPhone 6 Plus and i...,['[Compatible with Apple iPhone 6 Plus and iPh...,"['>#574,746 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,24,152,114
1041166,4,False,"07 16, 2016",A2KOKSKEDIBRGQ,B01HE0RNYO,NaN,Wise One,The covers my iPhone just fine. It is simple i...,Plain Jane of iPhone Covers,1468627200,0,NaN,['<b>Compatible with Apple iPhone 6 Plus and i...,['[Compatible with Apple iPhone 6 Plus and iPh...,"['>#574,746 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,28,233,159
1041167,3,False,"07 11, 2016",A1BO3NH4KKASUT,B01HE0RNYO,NaN,Bebescastle,This free case is okay it slim and trendy look...,Its easy to hold and not bulky,1468195200,0,NaN,['<b>Compatible

## Task 7 - Calculating rank number in Cell Phone and Accessories in rank column

### Running value_counts to see the variety of data in the rank column

In [52]:
df['rank'].value_counts()

[]                                                                                                                                                                                                                                                                                                                                                                         66796
['>#4,907 in Cell Phones & Accessories (See Top 100 in Cell Phones & Accessories)', '>#98 in Cell Phones & Accessories > Cell Phone Accessories > Car Accessories > Car Chargers', '>#435 in Electronics > Accessories & Supplies > Vehicle Electronics Accessories', '>#437 in Cell Phones & Accessories > Cell Phone Accessories > Chargers & Power Adapters']            1682
['>#6,626 in Cell Phones & Accessories (See Top 100 in Cell Phones & Accessories)', '>#141 in Electronics > Portable Audio & Video > MP3 & MP4 Player Accessories > Batteries & Battery Packs', '>#141 in Cell Phones & Accessories > Cell Phone Accessories > Batteri

#### Note - We can see that there are around 67000 rows which have no data for rank column (see below), but we are not dropping them as they still have reviewText which can very well be analyzed. We also note that various ranks have different number of digits in them, hence simply slicing the string from index 4 to say 9 won't work, so we need another approach.

In [53]:
df[df['rank'] == '[]']

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image,description,feature,rank,main_cat,imageURL,imageURLHighRes,no_of_sent,no_of_words,no_of_unique_words
530,4,False,"01 22, 2005",A1TTKKN4AY63QK,B00005UK9X,{'Package Type:': ' Standard Packaging'},Jan R. Klincewicz,"I drive a company-supplied vehicle, so alterat...",Rocks for normal drivers,1106352000,26,NaN,"[""Everything has its place and so does your St...","['Restraint System for your GPS Product', 'Kee...",[],Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,7,62,54
531,5,True,"06 25, 2003",A16PKR0PIB6959,B00005UK9X,{'Package Type:': ' Standard Packaging'},David Levine,I have a GPS V that I really like and this is ...,Great add-on,1056499200,14,NaN,"[""Everything has its place and so does your St...","['Restraint System for your GPS Product', 'Kee...",[],Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,7,62,52
532,5,True,"11 11, 2012",A1V4KTY1RXI606,B00005UK9X,{'Package Type:': ' Standard Packaging'},Barbara P.,As long as there are products like this availa...,A real grabber,1352592000,0,NaN,"[""Everything has its place and so does your St...","['Restraint System for your GPS Product', 'Kee...",[],Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,2,20,19
533,5,True,"08 31, 2009",A19RGDJPNZ7NZ,B00005UK9X,{'Package Type:': ' Standard Packaging'},MW,"I use it with a Garmin StreetPilot 7200, which...",Works with Garmin 7200,1251676800,0,NaN,"[""Everything has its place and so does your St...","['Restraint System for your GPS Product', 'Kee...",[],Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,7,82,71
534,5,True,"04 4, 2008",A100WO06OQR8BQ,B00005UK9X,{'Package Type:': ' Standard Packaging'},Duffer.,Works great. Easy to attach the GPS and easy t...,Great Device,1207267200,0,NaN,"[""Everything has its place and so does your St...","['Restraint System for your GPS Product', 'Kee...",[],Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,4,13,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1041154,1,True,"08 11, 2016",AKJK5FH23L8ZA,B01H72UJZ4,NaN,Rodolfo Tarelli,Over one milimiter uncovered in all the temper...,Really Bad Fit,1470873600,0,['https://images-na.ssl-images-amazon.com/imag...,[],[],[],Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,5,22,20
1041155,2,False,"08 3, 2016",A3TZH1A8PYUK1O,B01H72UJZ4,NaN,TuckerReviewer,"This is a clear glass protector, theoretically...",Not sized correctly.,1470182400,2,['https://images-na.ssl-images-amazon.com/imag...,[],[],[],Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,9,57,47
1041156,4,False,"08 2, 2016",ARKQFG8F2FJ8L,B01H72UJZ4,NaN,Amazon Customer,I've used quite a few screen protectors over y...,"Durable, a little narrow",1470096000,0,NaN,[],[],[],Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,19,137,108
1041157,3,False,"07 20, 2016",A24BH8L76QQFVD,B01H72UJZ4,NaN,PastorONeil,This is a tempered glass screen protector for ...,This is a bit annoying as dust and other parti...,1468972800,0,NaN,[],[],[],Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,17,157,101


### Approach

In [54]:
c = "['>#1,388,758 in Cell Phones & Accessories (See Top 100 in Cell Phones & Accessories)', '>#114,920 in Cell Phones & Accessories > Cases, Holsters & Clips']"
rank1 = ''
for i in range(len(c)):
    if c[i].isdigit() == True:
        rank1 += c[i]
    elif c[i] == 'i':
        break
rank1

'1388758'

#### Note - This rank is a string though, but can easily be converted into integer by type casting if required.

### Function to get rank for any element in the rank column

In [55]:
def rank_calc(rank_str):
    rank = ''
    for i in range(len(rank_str)):
        if rank_str[i].isdigit() == True:
            rank += rank_str[i]
        elif rank_str[i] == 'i':
            break
    return rank

### Example

In [56]:
rank_list1 = [rank_calc(df.iloc[i]['rank']) for i in range(23578,23590)]
rank_list1

['493361',
 '493361',
 '1393011',
 '1393011',
 '1393011',
 '1393011',
 '1393011',
 '1393011',
 '1393011',
 '1520743',
 '1520743',
 '1520743']

### Applying on main review data

In [57]:
start = time()

In [58]:
rank_list = [rank_calc(df.iloc[i]['rank']) for i in rows]

In [59]:
end = time()
end - start

102.1522114276886

In [60]:
rank_list

['228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '228544',
 '553803',
 '553803',
 '553803',
 '553803',
 '553803',
 '553803',
 '553803',
 '553803',
 '553803',
 '553803',
 '553803',
 '553803',
 '553803',
 '553803',
 '440155',
 '440155',
 '440155',
 '440155',
 '440155',
 '440155',
 '440155',
 '440155',
 '440155',
 '440155',
 '440155',

In [61]:
df['rank_no'] = rank_list

In [62]:
df

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image,description,feature,rank,main_cat,imageURL,imageURLHighRes,no_of_sent,no_of_words,no_of_unique_words,rank_no
0,5,True,"08 4, 2014",A24E3SXTC62LJI,7508492919,{'Color:': ' Bling'},Claudia Valdivia,Looks even better in person. Be careful to not...,Can't stop won't stop looking at it,1407110400,0,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,4,22,21,228544
1,5,True,"02 12, 2014",A269FLZCB4GIPV,7508492919,NaN,sarah ponce,When you don't want to spend a whole lot of ca...,1,1392163200,0,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,1,10,9,228544
2,3,True,"02 8, 2014",AB6CHQWHZW4TV,7508492919,NaN,Kai,"so the case came on time, i love the design. I...",Its okay,1391817600,0,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,3,22,21,228544
3,2,True,"02 4, 2014",A1M117A53LEI8,7508492919,NaN,Sharon Williams,DON'T CARE FOR IT. GAVE IT AS A GIFT AND THEY...,CASE,1391472000,0,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,3,5,5,228544
4,4,True,"02 3, 2014",A272DUT8M88ZS8,7508492919,NaN,Bella Rodriguez,"I liked it because it was cute, but the studs ...",Cute!,1391385600,0,NaN,"['', 'Elegani Butterfly Case 3D Pattern Back C...",['Fashionable with unique 3D butterfly design ...,"['>#228,544 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,2,12,12,228544
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1041164,5,True,"10 3, 2016",A3Q7GA2Q62E18Q,B01HE0RNYO,NaN,Kerry Moore,Excellent product for the price!!!,Inexpensive but good quality...,1475452800,0,NaN,['<b>Compatible with Apple iPhone 6 Plus and i...,['[Compatible with Apple iPhone 6 Plus and iPh...,"['>#574,746 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,2,3,3,574746
1041165,5,False,"07 18, 2016",AEKOIX49JGCUG,B01HE0RNYO,NaN,Wishart,I needed a new phone case for my iPhone 6 plus...,Great Case. Gonna Buy Again.,1468800000,0,NaN,['<b>Compatible with Apple iPhone 6 Plus and i...,['[Compatible with Apple iPhone 6 Plus and iPh...,"['>#574,746 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,24,152,114,574746
1041166,4,False,"07 16, 2016",A2KOKSKEDIBRGQ,B01HE0RNYO,NaN,Wise One,The covers my iPhone just fine. It is simple i...,Plain Jane of iPhone Covers,1468627200,0,NaN,['<b>Compatible with Apple iPhone 6 Plus and i...,['[Compatible with Apple iPhone 6 Plus and iPh...,"['>#574,746 in Cell Phones & Accessories (See ...",Cell Phones & Accessories,['https://images-na.ssl-images-amazon.com/imag...,['https://images-na.ssl-images-amazon.com/imag...,28,233,159,574746
1041167,3,False,"07 11, 2016",A1BO3NH4KKASUT,B01HE0RNYO,NaN,Bebescastle,This free case is okay it slim and trendy loo

## Task 8 - Exporting this dataframe as excel file (csv takes more space)

In [73]:
"""writer = pd.ExcelWriter(r"C:\Users\surya\Desktop\Jupyter Python Programs\Amazon Review NLP\Review_features_data.xlsx", engine='xlsxwriter',options={'strings_to_urls': False})
df.to_excel(writer, index=False)
writer.close()"""

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 28-29: truncated \UXXXXXXXX escape (<ipython-input-73-05ddcf0645ce>, line 1)